# Plan

``````
Web-3-Wallet-Generator/
├── .gitignore
├── LICENSE
├── README.md
├── docs/
│   └── index.md
├── tests/
│   └── test_wallet_generator.py
└── web_3_wallet_generator/
    └── __init__.py
    └── wallet_generator.py

``````

In [9]:
!pip install eth_account coincurve pysha3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.2/829.2 kB 20.4 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [56 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-x86_64-cpython-310
      copying sha3.py -> build/lib.macosx-10.9-x86_64-cpython-310
      running build_ext
      building '_pysha3' extension
      creating build/temp.macosx-10.9-x86_64-cpython-310
      creating build/temp.macosx-10.9-x86_64-cpython-310/Modules
      creating build/temp.macosx-10.9-x86_64-cpython-310/Modules/_sha3
      clang -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /Users/royceantonjose/anaconda3/envs/trading/include -fPIC -O2 -isystem /Users/royceantonjose/anaconda3/envs/trading/include -DPY_WITH_KECCAK=1 -I/Users/ro

# Ethereum Wallet

In [5]:
import os
import json
from eth_account import Account

def generate_wallet(blockchain='ethereum', wallet_name=None):
    # Generate a new account
    account = Account.create()

    # Get the private key and address
    wallet = {
        'private_key': account.privateKey.hex(),
        'address': account.address
    }

    # Create the directory if it doesn't exist
    directory = f'wallets/{blockchain}'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # If no wallet name is provided, generate a default one
    if wallet_name is None:
        wallet_number = 1
        wallet_name = f'wallet_{wallet_number}'
        while os.path.exists(f'{directory}/{wallet_name}.json'):
            wallet_number += 1
            wallet_name = f'wallet_{wallet_number}'

    # Save the wallet information to a JSON file
    with open(f'{directory}/{wallet_name}.json', 'w') as file:
        json.dump(wallet, file)

    return wallet

generate_wallet()

{'private_key': '0x4d1d2dac4e508635a379a87e6ca8cb32c303452b80106ca12a475cd7c07c1258',
 'address': '0x111F8ac6341C6832567C0FCbe0b0d09AeD974025'}

# Avalanche

In [10]:
import os
import json
import binascii
from coincurve import PrivateKey
from sha3 import keccak_256

def generate_wallet(blockchain='avalanche', wallet_name=None):
    # Generate a new private key
    private_key = PrivateKey()
    private_key_hex = private_key.to_hex()

    # Get the public key
    public_key = private_key.public_key.format(compressed=False)[1:]

    # Hash the public key and get the address
    address = '0x' + keccak_256(public_key).hexdigest()[-40:]

    # Create the wallet
    wallet = {
        'private_key': private_key_hex,
        'address': address
    }

    # Create the directory if it doesn't exist
    directory = f'wallets/{blockchain}'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # If no wallet name is provided, generate a default one
    if wallet_name is None:
        wallet_number = 1
        wallet_name = f'wallet_{wallet_number}'
        while os.path.exists(f'{directory}/{wallet_name}.json'):
            wallet_number += 1
            wallet_name = f'wallet_{wallet_number}'

    # Save the wallet information to a JSON file
    with open(f'{directory}/{wallet_name}.json', 'w') as file:
        json.dump(wallet, file)

    return wallet

generate_wallet(blockchain='avalanche')

{'private_key': 'ee17c5477c62f55ef90f62d3a1c0af01ed1f7a22d6b9c7c2f5b361dc784940b5',
 'address': '0x6d6f72251dfdb3e8fdf912b8359a77f19da5e113'}